In [1]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import torch
from torch import nn

from mvn.utils.minimon import MiniMon
from mvn.utils.misc import flush_cache

from mvn.ipynb import get_args, get_config, build_labels
from mvn.pipeline.setup import setup_dataloaders, setup_experiment, build_env
from mvn.pipeline.training import do_train

In [3]:
flush_cache()

is_distributed = False
master = True
device = torch.device(0)
print('using dev {}'.format(device))
    
args = get_args()

# just to get a feeling of the dataset
# labels, mask, indices = build_labels(config.dataset.train.labels_path, 10000)
# labels, mask, indices = build_labels(config.dataset.train.labels_path, 10, allowed_subjects=['S9', 'S11'])

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |
|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |
|---------------------------------------------------------------------------|
| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |

In [8]:
config = get_config(args)
do_train(None, None, config, device, is_distributed, master)

cam2cam model (res):
                    R_backbone has       2294 params (~ 50.6) %
                    t_backbone has       2243 params (~ 49.4) %
total params:                 4537
cam2cam estimation => adding 4537 params to grad ...
  training dataset length: 12
  validation dataset length: 5
Experiment name: human36m_alg_AlgebraicTriangulationNet@17.05.2021-11:24:26
#[do_train]: epoch    0 has started!
#[cam2cam]: training batch iter 0 losses: GEO ~ 7.985, TRANS ~ 206.689, POSE ~ 83420.664, ROTO ~ 8.806, 3D ~ 618.141, SELF R ~ 62.791, SELF t ~ 19.991, TOTAL ~ 720.913
#[cam2cam]: training batch iter 0 MPJPE: ~ 469.857 mm
#[cam2cam]: training batch iter 1 losses: GEO ~ 7.754, TRANS ~ 207.453, POSE ~ 86474.133, ROTO ~ 9.003, 3D ~ 622.078, SELF R ~ 64.502, SELF t ~ 20.330, TOTAL ~ 727.241
#[cam2cam]: training batch iter 1 MPJPE: ~ 466.954 mm
#[cam2cam]: training batch iter 2 losses: GEO ~ 7.977, TRANS ~ 206.810, POSE ~ 76947.594, ROTO ~ 8.818, 3D ~ 619.064, SELF R ~ 64.029, SELF t ~ 1